In [0]:
import os
os.chdir("drive/My Drive/oui")


In [0]:
import pandas as pd
import numpy as np


In [0]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
fre_input = fre.apply(lambda x: "<BOS> " + x.lower()+ " <EOS>")
fre = fre.apply(lambda x: x.lower() + " <EOS>")
eng = eng.apply(lambda x: x.lower())

def tokenize(data):
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(data)
  return tokenizer.texts_to_sequences(data), tokenizer
def pad_seq(sequences):
  padded = pad_sequences(sequences, padding = "post")
  return padded

In [0]:
eng_seq, eng_tok = tokenize(eng)
fre_input_seq, fre_tok = tokenize(fre_input)
fre_output_seq = fre_tok(fre)
eng_seq = pad_seq(eng_seq)
fre_output_seq = pad_seq(fre_output_seq)
fre_input_seq = pad_seq(fre_input_seq)
fre_output_seq = fre_output_seq.reshape(fre_output_seq.shape[0], fre_output_seq.shape[1], 1) #for sparse categorical crsooentropy
eng_max = eng_seq.shape[1]
fre_max = fre_input_seq.shape[1]
eng_vocab_len = len(eng_tok.word_indices)
fre_vocab_len = len(fre_tok.word_indices)
fre_idx = {id: word for word, id in fre_tok.word_index.items()}
index_to_words[0] = '<PAD>' 


In [0]:

LR = 1e-3
enc_input = Input(shape = (None,))
enc_emb = Embedding(eng_vocab_len, 64)(enc_input)
enc_lstm =  CuDNNLSTM(50, return_state = True)
_ , state_h, state_c = enc_lstm(emb)
enc_states = [state_h, state_c]

dec_inputs = Input(shape=(None,))
dec_emb = Embedding(fre_vocab_len, 64)(dec_input)
dec_lstm = CuSNNLSTM(50, return_sequences = True)
dec_out, _ = dec_lstm(dec_emb, initial_state = enc_states)
dec_soft = Dense(fre_vocab_len, activation = "softmax")
dec_pred = dec_soft(dec_out)

model = Model([enc_input, dec_input], dec_pred)

In [0]:
model.compile(optimizer = Adam(LR), loss = sparse_categorical_crossentropy, metrics = ["accuracy"])


In [0]:
model.fit([eng_seq, fre_input_seq], fre_output, batch_size = 128, epochs = 100, callbacks =[EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=50),mc = ModelCheckpoint('best_model.h5', monitor='val_loss', mode='min', save_best_only=True)])